In [19]:
import numpy as np
import pandas as pd
import import_ipynb
import pair_selection_DBSCAN_funclist_for_training as myFunc
import matplotlib.pyplot as plt

import math
import sklearn
import datetime

from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

In [20]:
price_df = pd.read_csv('/Users/yjban/Desktop/MFE/BAF646_Statistical_arbitrage/project/data/us_etf_price.csv')
etf_info = pd.read_csv('/Users/yjban/Desktop/MFE/BAF646_Statistical_arbitrage/project/data/etf_list/etfs_details_equity.csv')

In [38]:
start_date = '2020-01-01'
end_date = '2020-07-01'

test_start_date = '2020-07-01'
test_end_date = '2020-08-01'

pc_selecting_threshold = 0.9
eps = 1.8
min_samples = 4
cluster_size_limit = 100
cluster_member_counts = 100
inverse_threshold=-0.99
coint_pvalue_threshold=0.02
hurst_threshold=0.5
half_life_threshold=10
mean_reverting_freq=12

In [22]:
class Backtesting():

    def __init__(self, z_score_list, stop_loss, buy_z, sell_z, total_money=1000000):
        
        self.pairs_num = z_score_list.shape[0]
        self.total_time = z_score_list.shape[1]
        self.stop_loss = stop_loss
        self.buy_z = buy_z
        self.sell_z = sell_z
        
        self.inverse_price = np.zeros((2*self.pairs_num))
        self.total_stock = np.zeros((2*self.pairs_num))
        self.total_inverse = np.zeros((2*self.pairs_num))
        self.money_for_pair = int(total_money/self.pairs_num) * np.ones((self.pairs_num))
    
    def trade_decision(self, z_score, pairs_num):
        
        stop_loss = self.stop_loss
        buy_z = self.buy_z
        sell_z = self.sell_z
        total_stock = self.total_stock[2*(pairs_num)]
        total_inverse = self.total_inverse[2*(pairs_num)]
        stock, inverse = 0, 0 

        # 스탑로스 컷에 걸릴 때는 다 팔기
        if z_score >= stop_loss or z_score <= -1 * stop_loss:
            stock, inverse = -2, -2
        
        # buy threshold 이상이면서 해당 etf 보유하고 있지 않으면 매수
        elif z_score >= buy_z and total_stock==0:
            stock, inverse = 1, 0

        # sell threshold 이하이면서, 해당 etf 보유하고 있으면 etf 팔기
        elif z_score <= sell_z and total_stock > 0:
            stock, inverse = -1, 0

        # -buy threshold 이하로 떨어지는데, 인버스 etf 보유하고 있지 않을 땐 inverse etf 사기
        elif z_score <= -1 * buy_z and total_inverse==0:
            stock, inverse = 0, 1
        
        # -sell threshold 이상으로 올라가는데, 인버스 etf를 보유하고 있으면 inverse etf 팔기
        elif z_score >= -1 * sell_z and total_inverse > 0:
            stock, inverse = 0, -1 

        return stock, inverse
    
    def cal_trade_vol(self, stock_num, stock_signal, inv_signal, current_stock_price, current_inverse_price):

        trade_stock_vol = 0
        trade_inverse_vol = 0

        money_for_stock = self.money_for_pair[int(stock_num/2)] / 2

        if stock_signal < 0:
            trade_stock_vol = -1 * self.total_stock[stock_num]

        elif stock_signal > 0:
            trade_stock_vol = int(money_for_stock/current_stock_price)

        if inv_signal < 0:
            trade_inverse_vol = -1 * self.total_inverse[stock_num]
        
        elif inv_signal > 0:
            trade_inverse_vol = int(money_for_stock/current_inverse_price)

        return trade_stock_vol, trade_inverse_vol


    def cal_result(self, stock_num, stock_vol, inverse_vol, current_stock_price, current_inverse_price):

        # if self.total_inverse[stock_num] > 0:
        #     change_ratio = (current_price - prev_price) / prev_price
        #     self.inverse_price[stock_num] = (1 - change_ratio) * self.inverse_price[stock_num]


        if stock_vol > 0:
            self.total_stock[stock_num] += stock_vol
            self.money_for_pair[int(stock_num/2)] -= stock_vol * current_stock_price

        elif stock_vol < 0:
            self.total_stock[stock_num] += stock_vol
            self.money_for_pair[int(stock_num/2)] -= stock_vol * current_stock_price
        
            # Buy the inverse
        if inverse_vol > 0:
            self.total_inverse[stock_num] += inverse_vol
            self.money_for_pair[int(stock_num/2)] -= inverse_vol * current_inverse_price
            # self.inverse_price[stock_num] = current_price
        
        # Sell the inverse
        if inverse_vol < 0:
            self.total_inverse[stock_num] += inverse_vol
            self.money_for_pair[int(stock_num/2)] -= inverse_vol * current_inverse_price
            # self.money_for_pair[int(stock_num/2)] -= inverse_vol * self.inverse_price[stock_num]
            # self.inverse_price[stock_num] = 0

        # total_asset = (self.total_inverse[stock_num] * self.inverse_price[stock_num] + 
        #                 self.total_stock[stock_num] * current_price) 

        total_asset = (self.total_inverse[stock_num] * current_inverse_price +
                        self.total_stock[stock_num] * current_stock_price)

        return total_asset

    def backtesting(self, z_score_list, stock_price_list, inverse_price_list):

        total_asset = np.zeros((self.pairs_num, self.total_time))
        stock_a_vol_list = []
        inverse_a_vol_list = []
        stock_b_vol_list = []
        inverse_b_vol_list = []

        for t in range(self.total_time):
            for pair_num in range(self.pairs_num):
                stock_a_num = 2* pair_num
                stock_b_num = 2* pair_num + 1
                z_score = z_score_list[pair_num]

                # 매수매입 시그널
                stock_a, inverse_a = self.trade_decision(z_score[t], pair_num)
                stock_b = inverse_a
                inverse_b = stock_a

                # 매수매입 수량 결정
                stock_a_vol, inverse_a_vol = self.cal_trade_vol(stock_a_num, stock_a, inverse_a, stock_price_list[stock_a_num][t], inverse_price_list[stock_a_num][t])
                stock_b_vol, inverse_b_vol = self.cal_trade_vol(stock_b_num, stock_b, inverse_b, stock_price_list[stock_b_num][t], inverse_price_list[stock_b_num][t])
                stock_a_vol_list.append(stock_a_vol)
                inverse_a_vol_list.append(inverse_a_vol)
                stock_b_vol_list.append(stock_b_vol)
                inverse_b_vol_list.append(inverse_b_vol)

                # 투자 결과
                total_asset[pair_num][t] += self.cal_result(stock_a_num, stock_a_vol, inverse_a_vol, stock_price_list[stock_a_num][t], inverse_price_list[stock_a_num][t])
                total_asset[pair_num][t] += self.cal_result(stock_b_num, stock_b_vol, inverse_b_vol, stock_price_list[stock_b_num][t], inverse_price_list[stock_b_num][t])
                total_asset[pair_num][t] += self.money_for_pair[int(pair_num)]

        return total_asset, stock_a_vol_list, inverse_a_vol_list, stock_b_vol_list, inverse_b_vol_list

In [23]:
def get_pair(start_date, end_date):

    _, close_df, rtn_df, low_volume_etf = myFunc.preprocessing(price_df,etf_info, start_date, end_date)
    pc_rtn = myFunc.get_pca_return(rtn_df, pc_selecting_threshold)
    clusters_viz_list, clustered_series = myFunc.dbscan_clustering(close_df, pc_rtn, eps, min_samples, cluster_size_limit, cluster_member_counts)
    selected_pair, short_pair = myFunc.Pair_selection(close_df=close_df,
                                                rtn_df=rtn_df,
                                                low_volume_etf=low_volume_etf,
                                                clusters_viz_list=clusters_viz_list, 
                                                clustered_series=clustered_series, 
                                                inverse_threshold=inverse_threshold, 
                                                coint_pvalue_threshold=coint_pvalue_threshold, 
                                                hurst_threshold=hurst_threshold, 
                                                half_life_threshold=half_life_threshold, 
                                                mean_reverting_freq=mean_reverting_freq)

    return selected_pair, short_pair, close_df, rtn_df

def create_spread_function(pair_a, pair_b, start_date, end_date, alg='log'):

    def log_spread_func(pair_a, pair_b):
        
        spread = math.log(pair_b) - w_avg * math.log(pair_a)
        z_score = spread/w_std

        return (spread, z_score)

    def lr_spread_func(pair_a, pair_b):
        pair_a, pair_b = np.log(pair_a), np.log(pair_b)
        pair_a = pair_a * np.ones((1,1))
        poly = PolynomialFeatures(degree=best_degree)
        pair_a = poly.fit_transform(pair_a)

        spread = pair_b - model.predict(pair_a)
        z_score = spread / spread_std

        return (spread, z_score)

    target_a = np.log(pair_a[start_date:end_date])
    target_b = np.log(pair_b[start_date:end_date])

    # holding_period = end_date - start_date

    if alg == 'log':
        
        w_list = target_b / target_a
        w_avg = np.average(w_list)
        w_std = np.std(w_list)

        return log_spread_func
    
    elif alg == 'lr':

        min_cv_n = float('inf')
        best_degree = 0
        total_len = target_a.size

        permute_order = np.random.permutation(total_len)
        target_a = target_a[permute_order]
        target_b = target_b[permute_order]

        train_num = int(target_a.size/3*2)

        train_a = target_a[:train_num]
        train_b = target_b[:train_num]
        val_a = target_a[train_num:]
        val_b = target_b[train_num:]

        train_a = train_a.reshape(-1,1)
        val_a = val_a.reshape(-1,1)

        for degree in range(1,10,1):

            poly = PolynomialFeatures(degree=degree)
            poly_train_a = poly.fit_transform(train_a)
            poly_val_a = poly.fit_transform(val_a)

            model = linear_model.LassoCV(cv=5)
            model.fit(poly_train_a, train_b)
            
            mse = np.average((val_b - model.predict(poly_val_a))**2)

            if mse < min_cv_n:
                best_degree = degree
                min_cv_n = mse

        if best_degree == 0:
            print("error!")

        poly = PolynomialFeatures(degree= best_degree)
        poly_train_a = poly.fit_transform(train_a)
        model = linear_model.LassoCV(cv=5)
        model.fit(poly_train_a, train_b)

        b_pred = model.predict(poly_train_a)
        spread = train_b - b_pred
        spread_std = np.std(spread)

        return lr_spread_func

def gen_z_score_history(a, b, windows_width, spread_func_update_period):

    T = a.shape[0]
    z_score_list = np.zeros((T-windows_width))

    for t in range(T-windows_width):

        if t % spread_func_update_period==0:
            spread_func = create_spread_function(a,b,t,t+windows_width, 'lr')
        
        _, z_score = spread_func(a[t],b[t])
        z_score_list[t] = z_score

    return z_score_list

def get_best_threshold(z_score_list, stock_price_list, inverse_price_list, initial_money=1000000):
    stop_loss_candi = [1, 1.5, 2, 3]
    buy_z_candi = np.round(np.linspace(0, 2, 21), 1)
    sell_z_candi = np.round(np.linspace(0, 2, 21), 1)
    grid_search_result = {}
    for x in stop_loss_candi:
        for y in buy_z_candi:
            for z in sell_z_candi:
                stop_loss = x
                buy_z = y
                sell_z = z
                BT = Backtesting(z_score_list, total_money=initial_money, stop_loss=stop_loss, buy_z=buy_z, sell_z=sell_z)
                asset_per_pair, stock_a_vol_list, inverse_a_vol_list, stock_b_vol_list, inverse_b_vol_list = BT.backtesting(z_score_list, stock_price_list, inverse_price_list)
                total_asset = np.sum(asset_per_pair, axis=0)
                total_earning_ratio = total_asset[-1] / initial_money
                grid_search_result[(x,y,z)] = total_earning_ratio

    best_param = max(grid_search_result, key=grid_search_result.get)
    best_earning_ratio = grid_search_result[best_param]

    print('best_earning_ratio is: ', best_earning_ratio)

    return best_param

# -----------------------------------------------------------------------------------------------

In [24]:

selected_pair, short_pair, close_df, rtn_df = get_pair(start_date=start_date, end_date=end_date)

# 스크리닝 팩터를 통과한 페어가 없는 클러스터링은 제거






100%|██████████| 1338/1338 [00:13<00:00, 101.04it/s]


Clusters discovered: 3
Clusters formed: 2
Pairs to evaluate: 3438
final_clusters index :  [1, 2]


0it [00:00, ?it/s]/2 [00:00<?, ?it/s]
100%|██████████| 2/2 [00:40<00:00, 20.12s/it]


In [25]:
selected_pair_final = []
for i in range(len(selected_pair)):
    if len(selected_pair[i]) == 0:
        continue
    for j in range(len(selected_pair[i])):
        selected_pair_final.append(selected_pair[i][j])

In [26]:
selected_pair_final

[('EFZ', 'SEF'),
 ('SDP', 'FXP'),
 ('MYY', 'RWM'),
 ('DOG', 'SKF'),
 ('SKF', 'EWV'),
 ('EEV', 'SKF'),
 ('EPV', 'SKF'),
 ('YXI', 'SKF')]

In [27]:
# inverse etf list 뽑기
short_list = {}
selected_pair_etf = []
for i in range(len(selected_pair_final)):
    selected_pair_etf.append(selected_pair_final[i][0])
    selected_pair_etf.append(selected_pair_final[i][1])
    short_list[selected_pair_final[i][0]] = rtn_df.corr()[selected_pair_final[i][0]].idxmin()
    short_list[selected_pair_final[i][1]] = rtn_df.corr()[selected_pair_final[i][1]].idxmin()

In [28]:

# 모델 트레이닝을 위한 포메이션 기간동안의 페어 가격 추출
training_set_price = pd.DataFrame(index=close_df.index)
for i in range(len(selected_pair_final)):
    training_set_price = pd.concat([training_set_price, close_df[list(selected_pair_final[i])]], axis=1)

training_set_inverse_price = close_df[[short_list[x] for x in training_set_price.columns]]

# 페아별 z-score 뽑기
for i in range(len(selected_pair_final)):

    print("pairs = (" + str(selected_pair_final[i]) + ")\n")

    a = training_set_price.iloc[:,2*i].to_numpy()
    b = training_set_price.iloc[:,2*i+1].to_numpy()

    spread_func = create_spread_function(a, b, 0, -1, alg='lr')
    x = np.arange(len(a))
    z_score_history = np.zeros((len(a)))

    for j in range(len(a)):
        (spread, z_score_history[j]) = spread_func(a[j], b[j])

# 패어별로 z-score time series 뽑기
z_score_history_list = []

for i in range(len(selected_pair_final)):

    price_history = training_set_price.to_numpy()[:,2*i:2*i+2].T
    z_score_history = gen_z_score_history(price_history[0], price_history[1], 20, 20).reshape(1, -1)
    z_score_history_list.append(z_score_history)

z_score_history_list = np.array(z_score_history_list).reshape(len(selected_pair_final), -1)
z_score_list = z_score_history_list[:len(selected_pair_final),:]


pairs = (('EFZ', 'SEF'))

pairs = (('SDP', 'FXP'))

pairs = (('MYY', 'RWM'))

pairs = (('DOG', 'SKF'))

pairs = (('SKF', 'EWV'))

pairs = (('EEV', 'SKF'))

pairs = (('EPV', 'SKF'))

pairs = (('YXI', 'SKF'))



In [29]:
stock_price_list = training_set_price.T.to_numpy()[:,20:]
inverse_price_list = training_set_inverse_price.T.to_numpy()[:,20:]

In [30]:
# best threshold 찾기
best_threshold = {}
for i in range(len(selected_pair_final)):
    best_threshold[selected_pair_final[i]] = get_best_threshold(z_score_list[i].reshape(1,-1),stock_price_list, inverse_price_list)

best_earning_ratio is:  1.309012551858901
best_earning_ratio is:  1.2838104409751896
best_earning_ratio is:  1.1332627728939055
best_earning_ratio is:  1.5737646200370778
best_earning_ratio is:  1.2101382838935852
best_earning_ratio is:  1.3978045882358552
best_earning_ratio is:  1.2914852244548798
best_earning_ratio is:  1.3897330910072332


In [31]:
short_list

{'EFZ': 'EFA',
 'SEF': 'UYG',
 'SDP': 'IDU',
 'FXP': 'FXI',
 'MYY': 'IJH',
 'RWM': 'URTY',
 'DOG': 'DIA',
 'SKF': 'UYG',
 'EWV': 'JPXN',
 'EEV': 'EEM',
 'EPV': 'IEV',
 'YXI': 'FXI'}

In [41]:
# 테스트 기간 데이터 로딩
_, close_df_test, rtn_df_test, low_volume_etf_test = myFunc.preprocessing(price_df, etf_info, test_start_date, test_end_date)

test_set_price = pd.DataFrame(index=close_df_test.index)
for i in range(len(selected_pair_final)):
    test_set_price = pd.concat([test_set_price, close_df_test[list(selected_pair_final[i])]], axis=1)

test_set_inverse_price = close_df_test[[short_list[x] for x in test_set_price.columns]]

100%|██████████| 1416/1416 [00:08<00:00, 170.34it/s]


In [42]:
test_set_price

,EFZ,SEF,SDP,FXP,MYY,RWM,DOG,SKF,SKF,EWV,EEV,SKF,EPV,SKF,YXI,SKF
Date,,,,,,,,,,,,,,,,
2020-07-01,25.049999,20.139999,23.760000,50.139999,39.610001,36.509998,47.910000,51.279999,51.279999,23.459999,31.940001,51.279999,25.250000,51.279999,17.870001,51.279999
2020-07-02,24.770000,20.110001,23.799999,46.830002,39.480000,36.360001,47.709999,51.119999,51.119999,23.139999,30.480000,51.119999,24.719999,51.119999,17.290001,51.119999
2020-07-06,24.379999,19.840000,24.260000,37.970001,39.029999,36.099998,46.860001,49.720001,49.720001,22.500000,27.870001,49.720001,23.709999,49.720001,15.620000,49.720001
2020-07-07,24.719999,20.280001,24.400000,39.959999,39.730000,36.730000,47.540001,51.720001,51.720001,22.900000,28.770000,51.720001,24.430000,51.720001,16.059999,51.720001
2020-07-08,24.510000,20.110001,24.080000,37.470001,39.500000,36.450001,47.250000,51.119999,51.119999,22.959999,27.330000,51.119999,23.870001,51.119999,15.560000,51.119999
2020-07-09,24.809999,20.510000,24.719999,37.980000,40.070000,37.200001,47.880001,53.000000,53.000000,23.160000,27.299999,53.000000,24.570000,53.000000,15.660000,53.000000
2020-07-10,24.530001,19.980000,23.799999,39.060001,39.520000,36.590000,47.230000,50.480000,50.480000,22.530001,27.660000,50.480000,24.049999,50.480000,15.880000,50.480000
2020-07-13,24.670000,20.059999,23.700001,39.990002,39.889999,37.049999,47.160000,50.840000,50.840000,22.780001,27.980000,50.840000,24.330000,50.840000,16.070000,50.840000
2020-07-14,24.340000,19.889999,23.219999,40.580002,39.299999,36.419998,46.169998,49.840000,49.840000,22.430000,27.900000,49.840000,23.430000,49.840000,16.209999,49.840000


In [43]:
z_score_history_test_list = []
for i in range(len(selected_pair_final)):

    print("pairs = (" + str(selected_pair_final[i]) + ")\n")

    a = test_set_price.iloc[:,2*i].to_numpy()
    b = test_set_price.iloc[:,2*i+1].to_numpy()

    spread_func = create_spread_function(a, b, 0, -1, alg='lr')
    x = np.arange(a.shape[0]).reshape(1,-1)
    z_score_history_test = np.zeros((a.shape[0]))

    for j in range(a.shape[0]):
        (spread, z_score_history_test[j]) = spread_func(a[j], b[j])

    z_score_history_test_list.append(z_score_history_test)

z_score_list_test = np.array(z_score_history_test_list).reshape(len(selected_pair_final), -1)

stock_price_list_test = test_set_price.T.to_numpy()
inverse_price_list_test = test_set_inverse_price.T.to_numpy()


print(selected_pair_final)

pairs = (('EFZ', 'SEF'))

pairs = (('SDP', 'FXP'))

pairs = (('MYY', 'RWM'))

pairs = (('DOG', 'SKF'))

pairs = (('SKF', 'EWV'))

pairs = (('EEV', 'SKF'))

pairs = (('EPV', 'SKF'))

pairs = (('YXI', 'SKF'))

[('EFZ', 'SEF'), ('SDP', 'FXP'), ('MYY', 'RWM'), ('DOG', 'SKF'), ('SKF', 'EWV'), ('EEV', 'SKF'), ('EPV', 'SKF'), ('YXI', 'SKF')]


In [44]:
# 트레이딩 결과 
performance = {}
for i in range(len(selected_pair_final)):

    initial_money = 1000000
    stop_loss = best_threshold[selected_pair_final[i]][0]
    buy_z = best_threshold[selected_pair_final[i]][1]
    sell_z = best_threshold[selected_pair_final[i]][2]

    BT = Backtesting(z_score_list_test, total_money=initial_money, stop_loss=stop_loss, buy_z=buy_z, sell_z=sell_z)
    asset_per_pair_test, stock_a_vol_list_test, inverse_a_vol_list_test, stock_b_vol_list_test, inverse_b_vol_list_test = BT.backtesting(z_score_list_test, stock_price_list_test, inverse_price_list_test)

    if len([x for x in stock_a_vol_list_test if x != 0])==0 and len([x for x in stock_b_vol_list_test if x != 0])==0:
        performance[selected_pair_final[i]] = np.nan

    else:
        total_asset_test = np.sum(asset_per_pair_test, axis=0)
        total_earning_ratio_test = total_asset_test[-1] / initial_money
        performance[selected_pair_final[i]] = total_earning_ratio_test

In [45]:
performance

{('EFZ', 'SEF'): 1.0086250488739015,
 ('SDP', 'FXP'): 1.0174846619615554,
 ('MYY', 'RWM'): 1.0176877505340576,
 ('DOG', 'SKF'): 1.0246854398202896,
 ('SKF', 'EWV'): 1.0159008913898468,
 ('EEV', 'SKF'): 1.0239580038185119,
 ('EPV', 'SKF'): 1.0314566290569305,
 ('YXI', 'SKF'): 1.0210438303985596}